In [282]:
%matplotlib inline

import numpy as np
import pandas as pd
import re

import Bio.PDB
import Bio.SeqIO
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist
from Bio.SeqUtils import seq3

In [283]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
# launch pymol

import __main__
__main__.pymol_argv = ['pymol','-Qc'] # Pymol: quiet and no GUI
from time import sleep
import pymol
from pymol import cmd

pymol.finish_launching()

In [299]:
# paths

hla_fasta = '../data/mhc_alleles/hla/hla_prot.fasta'
pdb_path = '../data/structures'
aligned_pdb_path = '../sandbox/aligned_pdb'

In [285]:
# tables

annotation = pd.read_csv('../sandbox/detailed_mhc', sep='\t')
alleles_vs_pdb = pd.read_csv('../sandbox/alleles_vs_pdb', sep='\t', index_col=0, low_memory=False)
peptides_vs_pdb = pd.read_csv('../sandbox/peptides_vs_pdb', sep='\t', index_col=0, low_memory=False)
ligand_assays = pd.read_csv('../data/epitopes/mhc_ligand_full.csv', skiprows=1, low_memory=False)

In [286]:
# table conversion

ligand_assays = ligand_assays[ligand_assays['MHC allele class'] == 'I']
affinity = ligand_assays[['Description', 'Allele Name', 'Qualitative Measure', 'Quantitative measurement']]
affinity['Length'] = [len(x) for x in affinity.Description]
affinity = affinity[(affinity.Length < 12) & (affinity.Length > 7)]
affinity = affinity[affinity['Allele Name'].str.contains('HLA-[ABC]\*\S+$', regex=True)]
affinity['Allele Name'] = [x[4:] for x in affinity['Allele Name']]

iclass = annotation[annotation.mhc_class == 'I']
iclass = iclass[iclass.a_chain_allele.str.contains('[ABC]\*', regex=True)]
iclass.a_chain_allele = [':'.join(x.split(':')[:2]) for x in iclass.a_chain_allele]
iclass = iclass.reset_index(drop=True)

In [292]:
# place mhc to the specific position
def localize_pdb(name, amhc, bmhc, pept, plen):
    # define peptide CA ids
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb = Bio.PDB.CaPPBuilder() 
    struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
    reslist = ppb.build_peptides(struct[pept])[0]
    firstCA = reslist[0].get_id()[1]
    lastCA = reslist[-1].get_id()[1]
    
    assert(lastCA-firstCA+1 == plen)
    
    # translate complex by -startCA
    cmd.select('sele', '%s//%s/%i/CA %s//%s/%i/CA' % (name, pept, firstCA, name, pept, lastCA))
    start, end = cmd.get_coords('sele', 1)
    cmd.translate(list(-start), name)
    
    # rotate complex around (0, 0, 0), so that endCA.y = 0
    start, end = cmd.get_coords('sele', 1)
    angle = np.arctan(end[1]/end[0])
    if end[0] < 0.0:
        angle = np.pi + angle
    angle = -angle*180.0/np.pi
    cmd.rotate('z', angle, name, origin=[0.0, 0.0, 0.0])
    
    # superpose peptide line on x axis
    start, end = cmd.get_coords('sele', 1)
    angle = np.arctan(end[2]/end[0])
    if end[0] < 0.0:
        angle = np.pi + angle
    angle = angle*180.0/np.pi
    print(cmd.rotate('y', angle, name, origin=[0.0, 0.0, 0.0]))
    
    # shift along x axis
    start, end = cmd.get_coords('sele', 1)
    print(cmd.translate([-(start[0]+end[0])/2.0, 0.0, 0.0], name))
    
    # superpose com on the xz plane
    com = cmd.centerofmass('%s//%s// %s//%s//' % (name, amhc, name, bmhc))
    angle = np.arctan(com[2]/com[1])
    if com[1] < 0.0:
        angle = np.pi + angle
    angle = -(angle)*180.0/np.pi
    print(cmd.rotate('x', angle, name, origin=[0.0, 0.0, 0.0]))
    
    # verify the result
    com = cmd.centerofmass('%s//%s// %s//%s//' % (name, amhc, name, bmhc))
    start, end = cmd.get_coords('sele', 1)
    assert(com[2] < 0.001)
    assert(start[2] < 0.001 and start[1] < 0.001)
    assert(end[2] < 0.001 and end[1] < 0.001)
    
    #cmd.delete('!(%s//%s// %s//%s// %s//%s//)' % (name, mhca, name, mhcb, name, pept))
    
    #cmd.save('../sandbox/positioned_pdb/%s_posed.pdb' % name, 
    #         '%s//%s// %s//%s// %s//%s//' % (name, mhca, name, mhcb, name, pept))
    
    sleep(0.5)

# name should be preloaded to pymol with cmd.load()    
def localize_pdb_wrapper(name, table):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    plen = int(line['antigen_len'])

    localize_pdb(name, mhca, mhcb, pept, plen)
    
# name should be preloaded to pymol with cmd.load()    
def clean(name, table):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    
    cmd.delete('!(%s//%s// %s//%s// %s//%s//)' % (name, mhca, name, mhcb, name, pept))
    
# reference and mobile should be preloaded to pymol with cmd.load()
def align_mob2ref(mob_name, ref_name, table):
    line = table[table.pdb == ref_name].iloc[0, :]
    ref_mhca = line['a_chain_id']
    ref_mhcb = line['b_chain_id']
    ref_pept = line['antigen_id']
    
    line = table[table.pdb == mob_name].iloc[0, :]
    mob_mhca = line['a_chain_id']
    mob_mhcb = line['b_chain_id']
    mob_pept = line['antigen_id']

    cmd.align('%s//%s// %s//%s//' % (mob_name, mob_mhca, mob_name, mob_pept), 
              '%s//%s// %s//%s//' % (ref_name, ref_mhca, ref_name, ref_pept))
    
# name should be preloaded to pymol with cmd.load()
def save2pdb(name, table, path):
    line = table[table.pdb == name].iloc[0, :]
    mhca = line['a_chain_id']
    mhcb = line['b_chain_id']
    pept = line['antigen_id']
    
    cmd.save(path, '%s//%s// %s//%s// %s//%s//' % (name, mhca, name, mhcb, name, pept)) 

In [300]:
def align_all_pdbs(table):
    ref=table.loc[0, 'pdb']

    cmd.reinitialize()
    cmd.load(pdb_path+'/'+ref+'.pdb', ref)
    clean(ref, table)
    localize_pdb_wrapper(ref, table)

    for i in range(1, table.shape[0]):
        name=table.loc[i, 'pdb']
        cmd.load(pdb_path+'/'+name+'.pdb', name)
        clean(name, table)
        align_mob2ref(name, ref, table)
        save2pdb(name, table, '%s/%s_aligned.pdb' % (aligned_pdb_path, name))
        cmd.delete(name)

    save2pdb(ref, table, '%s/%s_aligned.pdb' % (aligned_pdb_path, ref))
    cmd.delete(ref)
        
#align_all_pdbs(iclass)

In [336]:
# s1 - mutated sequence
# s2 - reference sequence
def align(s1, s2):
    matrix = matlist.blosum62
    gap_open = -100.0
    gap_extend = -0.5
    alns = pairwise2.align.globalds(s1, s2, matrix, gap_open, gap_extend)
    top_aln = alns[0]
    
    
    print(top_aln[0])
    print(top_aln[1])
    
    # check that there is no gaps inside the alignment
    assert(re.search('[A-Z]-+[A-Z]', top_aln[0]) == None)
    assert(re.search('[A-Z]-+[A-Z]', top_aln[1]) == None)
    
    mutations = []
    gap = re.match('(-+)', top_aln[0])
    shift = 0
    if gap:
        shift = gap.end(1) - gap.start(1)
        
    for i, a1, a2 in zip(list(range(-shift, len(top_aln[0])-shift)), top_aln[0], top_aln[1]):
        if a1 != '-' and a2 != '-':
            if a1 != a2:
                mutations.append((i, a1, a2))
    return mutations

# mutate pdb structure in accordance to input sequences
# here we assume that all chains are continuous: CHECK THIS
def mutate(pdb, newseq, newpep, table):
    parser = Bio.PDB.PDBParser(QUIET=True)
    ppb=Bio.PDB.PPBuilder()
    
    name = pdb; 
    amhc = table[table.pdb == pdb].iloc[0, 3]
    pept = table[table.pdb == pdb].iloc[0, 7]
    b2m = table[table.pdb == pdb].iloc[0, 5]
    
    struct = parser.get_structure(id=name, file=pdb_path+'/'+name+'.pdb')[0]
    oldseq = ''.join([str(x.get_sequence()) for x in ppb.build_peptides(struct[amhc])])
    oldpep = table[table.pdb == pdb].iloc[0, 8]
    
    assert(len(oldpep) == len(newpep))
    mhc_mutations = align(oldseq, newseq)
    pep_mutations = align(oldpep, newpep)
    
    # remove alternative atom locations
    cmd.remove("not alt ''+A")
    cmd.alter('all', "alt=''")
    #cmd.save('../sandbox/mutated/original.pdb', '%s//%s// %s//%s// %s//%s//' % (name, amhc, name, pept, name, b2m))
    
    # mutate
    cmd.wizard('mutagenesis')
    cmd.refresh_wizard()

    first_aa_id = ppb.build_peptides(struct[amhc])[0][0].get_id()[1]
    for i, old, new in mhc_mutations:
        print(i, old, new)
        cmd.get_wizard().do_select(amhc+'/'+str(i+first_aa_id)+'/')
        cmd.get_wizard().set_mode(seq3(new).upper())
        cmd.get_wizard().apply()
    print("\n%s: %i MHC mutations\n" % (name, len(mhc_mutations)))
        
    first_aa_id = ppb.build_peptides(struct[pept])[0][0].get_id()[1]
    for i, old, new in pep_mutations:
        print(i, old, new)
        cmd.get_wizard().do_select(pept+'/'+str(i+first_aa_id)+'/')
        cmd.get_wizard().set_mode(seq3(new).upper())
        cmd.get_wizard().apply()
    print("\n%s: %i peptide mutations\n" % (name, len(pep_mutations)))
        
    cmd.set_wizard()
    #cmd.save('../sandbox/mutated/mutated.pdb', '%s//%s// %s//%s// %s//%s//' % (name, amhc, name, pept, name, b2m))
    
    #cmd.delete(name)
    sleep(0.5)

# wrapper for mutate(..)
def find_and_mutate(pepseq, allele, alleledict, pep2pdb, mhc2pdb, pdb_table, first_best=10):
    chunk = mhc2pdb.loc[mhc2pdb.index.str.startswith(allele), :]
    best_pdbs = (chunk.apply(min) + pep2pdb.loc[pepseq, :]/10).sort_values()
    
    counter = 0
    for i in xrange(len(best_pdbs)):
        try:
            best_pdb = best_pdbs.index[i]
            allele_list = chunk.loc[:, best_pdb]
            full_allele = allele_list[allele_list == min(allele_list)].index[0]
            print(full_allele)
            alleleseq = str(alleledict[full_allele].seq)
            print('Compability score: %f' % best_pdbs[best_pdb])
            
            cmd.reinitialize()    
            cmd.load(aligned_pdb_path+'/'+best_pdb+'_aligned.pdb', best_pdb)
            cmd.save('../sandbox/mutated/original%i.pdb' % counter, best_pdb)
            mutate(best_pdb, alleleseq, pepseq, pdb_table)
            cmd.save('../sandbox/mutated/mutated%i.pdb' % counter, best_pdb)
            cmd.delete(best_pdb)
            
        except AssertionError:
            print('Assertion error: %s, %s, %s' % (pepseq, allele, best_pdb))
            continue
    
        else:
            if counter == first_best-1:
                break
            else:
                counter += 1

In [337]:
pepseq

'SPGMVPLHI'

In [338]:
# allele dictionary
alleledict = Bio.SeqIO.to_dict(Bio.SeqIO.parse(hla_fasta, 'fasta'))
for key in alleledict.keys():
    alleledict[key.split(';')[1]] = alleledict.pop(key)
    
idx = 100000
pepseq = affinity.iloc[idx, 0]
allele = affinity.iloc[idx, 1]
find_and_mutate(pepseq, allele, alleledict, peptides_vs_pdb, alleles_vs_pdb, iclass)

In [68]:
iclass.head()

,pdb,mhc_class,host,a_chain_id,a_chain_allele,b_chain_id,b_chain_allele,antigen_id,antigen_seq,antigen_len,antigen_name,antigen_organism,comments
0,3i6l,I,Human,D,A*24:02,E,B2M,F,QFKDNVILL,9,Nucleoprotein,SARS coronavirus Tor2,NaN
1,5swq,I,Human,A,A*02:01,B,B2M,C,CVNGSCFTV,9,neuraminidase,Influenza A virus,NaN
2,3x14,I,Human,A,B*08:02,B,B2M,C,FLRGRAYGL,9,nuclear antigen EBNA-3,Human herpesvirus 4,NaN
3,4jqx,I,Human,A,B*44:03,C,B2M,B,EECDSELEIKRY,12,Trans-activator protein BZLF1,Human herpesvirus 4,NaN
4,5c0d,I,Human,A,A*02:01,B,B2M,C,AQWGPDPAAA,10,NaN,NaN,NaN


In [277]:
def set_values(matrix, atom_desc, left_bottom, right_up, bin_size):
    aas = ['GLY', 'ALA', 'ILE', 'LEU', 
           'PRO', 'SER', 'THR', 'CYS', 
           'MET', 'ASP', 'ASN', 'GLU', 
           'GLN', 'LYS', 'ARG', 'HIS', 
           'PHE', 'TYR', 'TRP']
    
    x = atom_desc[5]
    y = atom_desc[6]
    z = atom_desc[7]
    
    # check if the atom is inside of the box
    if (x-left_bottom[0] < 0.0 or y-left_bottom[1] < 0.0 or z-left_bottom[2] < 0.0):
        return 0
    
    if (x-right_up[0] > 0.0 or y-right_up[1] > 0.0 or z-right_up[2] > 0.0):
        return 0
    
    # compute atom index
    x_bin = np.floor((x-left_bottom[0])/bin_size)
    y_bin = np.floor((y-left_bottom[1])/bin_size)
    z_bin = np.floor((z-left_bottom[2])/bin_size)
    
    # increase values in matrix corresponding to the atom
    if atom_desc[0] != 'ATOM':
        matrix['X'][x_bin, y_bin, z_bin] += 1.0
        matrix['XXX'][x_bin, y_bin, z_bin] += 1.0
        return 1
    
    if atom_desc[1] == 'CA':
        matrix['CA'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[2] in ['C', 'N', 'O', 'S']:
        matrix[atom_desc[2]][x_bin, y_bin, z_bin] += 1.0
    else:
        matrix['X'][x_bin, y_bin, z_bin] += 1.0
    
    if atom_desc[3] in aas:
        matrix[atom_desc[3]][x_bin, y_bin, z_bin] += 1.0
    else:
        matrix['XXX'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[4] == 'H':
        matrix['HELIX'][x_bin, y_bin, z_bin] += 1.0
        
    if atom_desc[4] == 'S':
        matrix['STRAND'][x_bin, y_bin, z_bin] += 1.0
        
    return 2

# convert complex to 4d matrix
# name should be preloaded to pymol with cmd.load()
def mhc2matrix(name, left_bottom=(-10.0, -10.0, -10.0), right_up=(10.0, 10.0, 10.0), bin_size=1.0):
    x_bins = int((right_up[0]-left_bottom[0])/bin_size)
    y_bins = int((right_up[1]-left_bottom[1])/bin_size)
    z_bins = int((right_up[2]-left_bottom[2])/bin_size)
    
    layer_names = ['CA', 'C', 'N', 'O', 'S', 'X', 
                   'GLY', 'ALA', 'ILE', 'LEU', 
                   'PRO', 'SER', 'THR', 'CYS', 
                   'MET', 'ASP', 'ASN', 'GLU', 
                   'GLN', 'LYS', 'ARG', 'HIS', 
                   'PHE', 'TYR', 'TRP', 'XXX', 
                   'HELIX', 'STRAND']
    
    filter_num = len(layer_names)
    m = np.zeros((x_bins, y_bins, z_bins), dtype=zip(layer_names, [np.float32]*len(layer_names)))
    
    atoms = []
    myspace = {'atoms': atoms}
    cmd.iterate_state(0, name, 'atoms.append([type, name, elem, resn, ss, x, y, z])', space=myspace)
    #print(atoms)
    
    for atom_desc in atoms: #cmd.iterate_state(0, name+'//C//', 'type, name, elem, resn, ss, x, y, z'):
        set_values(m, atom_desc, left_bottom, right_up, bin_size)
        
    return np.array([m[name] for name in layer_names])